# SMA Crossover Strategy with Martingale
Author: Mohamed Abbas El kayal

## Disclaimer
Trading the financial markets imposes a risk of financial loss. TraderPy is not responsible for any financial losses that viewers suffer. Content is educational only and does not serve as financial advice. Information or material is provided ‘as is’ without any warranty. 

Past trading results do not indicate future performance. Strategies that worked in the past may not reflect the same results in the future.

In [810]:
import datetime
import MetaTrader5 as mt5
import pandas as pd
import plotly.express as px
import numpy as np

from datetime import datetime
from IPython.display import display

In [811]:
# connect to MetaTrader5 as mt5
mt5.initialize()

True

In [812]:
# settings
symbol = 'EURUSD'
timeframe = mt5.TIMEFRAME_H4
start_pos = 0
num_bars = 2200

fast_sma_period = 5
slow_sma_period = 200

In [813]:
# Requesting historical data from MetaTrader 5
bars = mt5.copy_rates_from_pos(symbol, timeframe, start_pos, num_bars)

# Converting bars we got from MetaTrader5 into dataframe
df = pd.DataFrame(bars)[['time', 'open', 'high', 'low', 'close', 'spread']]

#  Re-factoring time format into human readable
df['time'] = pd.to_datetime(df['time'], unit='s')


In [814]:
#  define the indicators
def indicator_sma (dataframe):
    dataframe['fast_sma'] = dataframe['close'].rolling(fast_sma_period).mean()
    dataframe['slow_sma'] = dataframe['close'].rolling(slow_sma_period).mean()

    # finding crossovers
    dataframe['prev_fast_sma'] = dataframe['fast_sma'].shift(1)

    dataframe.dropna(inplace=True)

indicator_sma(dataframe=df)
df

,time,open,high,low,close,spread,fast_sma,slow_sma,prev_fast_sma
199,2022-10-12 08:00:00,0.97144,0.97321,0.96945,0.97028,20,0.971776,0.989081,0.972098
200,2022-10-12 12:00:00,0.97029,0.97191,0.96860,0.97043,20,0.970572,0.988952,0.971776
201,2022-10-12 16:00:00,0.97043,0.97103,0.96668,0.96824,19,0.970114,0.988799,0.970572
202,2022-10-12 20:00:00,0.96823,0.97194,0.96798,0.97014,19,0.970098,0.988634,0.970114
203,2022-10-13 00:00:00,0.97012,0.97212,0.96968,0.97135,18,0.970088,0.988501,0.970098
...,...,...,...,...,...,...,...,...,...
2195,2024-01-25 00:00:00,1.08833,1.08855,1.08692,1.08766,18,1.089810,1.092355,1.089498
2196,2024-01-25 04:00:00,1.08768,1.08810,1.08735,1.08773,17,1.089318,1.092412,1.089810
2197,2024-01-25 08:00:00,1.08773,1.08987,1.08748,1.08949,16,1.088734,1.092476,1.089318
2198,2024-01-25 12:00:00,1.08950,1.09006,1.08657,1.08782,18,1.088210,1.092535,1.088734


In [815]:
def create_signal(dataframe):
    
    conditions_signal = [
        # dataframe['fast_sma'].gt(dataframe['slow_sma']) & dataframe['prev_fast_sma'].lt(dataframe['slow_sma']), # Buy Signal
        # dataframe['fast_sma'].lt(dataframe['slow_sma']) & dataframe['prev_fast_sma'].gt(dataframe['slow_sma']), # Sell Signal
        dataframe['fast_sma'].gt(dataframe['slow_sma']), # up signal
        dataframe['fast_sma'].lt(dataframe['slow_sma']), # down signal
    ]

    choices_signal = [
        # 'buy',
        # 'sell', 
        'buy', 
        'sell'
        ]

    dataframe['signal'] = np.select(conditions_signal, choices_signal, default=None)

    # export dataframe to excel file

    # create file name based on current date and time
    # filename = datetime.now().strftime('%Y-%m-%d-%H-%M-%S.xlsx')

    # export dataframe to excel file
    # df.to_excel(filename)  

    dataframe['signal'] = dataframe['signal'].shift(1)
    
    return dataframe

create_signal(dataframe=df)

,time,open,high,low,close,spread,fast_sma,slow_sma,prev_fast_sma,signal
199,2022-10-12 08:00:00,0.97144,0.97321,0.96945,0.97028,20,0.971776,0.989081,0.972098,NaN
200,2022-10-12 12:00:00,0.97029,0.97191,0.96860,0.97043,20,0.970572,0.988952,0.971776,sell
201,2022-10-12 16:00:00,0.97043,0.97103,0.96668,0.96824,19,0.970114,0.988799,0.970572,sell
202,2022-10-12 20:00:00,0.96823,0.97194,0.96798,0.97014,19,0.970098,0.988634,0.970114,sell
203,2022-10-13 00:00:00,0.97012,0.97212,0.96968,0.97135,18,0.970088,0.988501,0.970098,sell
...,...,...,...,...,...,...,...,...,...,...
2195,2024-01-25 00:00:00,1.08833,1.08855,1.08692,1.08766,18,1.089810,1.092355,1.089498,sell
2196,2024-01-25 04:00:00,1.08768,1.08810,1.08735,1.08773,17,1.089318,1.092412,1.089810,sell
2197,2024-01-25 08:00:00,1.08773,1.08987,1.08748,1.08949,16,1.088734,1.092476,1.089318,sell
2198,2024-01-25 12:00:00,1.08950,1.09006,1.08657,1.08782,18,1.088210,1.092535,1.088734,sell


In [816]:
def create_ohlcv(dataframe):
  deals = dataframe.groupby((dataframe.signal != dataframe.signal.shift()).cumsum(), as_index= False).agg(
    signal = ('signal', 'first'),
    open_price = ('open', 'first'),
    highest = ('high', 'max'),
    lowest = ('low', 'min')
  )

  safety = 0.97
  deals['open_price'] = deals['open_price'] * safety
  deals['close_price'] = deals['open_price'].shift(-1)

  deals.dropna(inplace=True)
  
  return deals

deals = create_ohlcv(dataframe=df)

deals

,signal,open_price,highest,lowest,close_price
1,sell,0.941181,0.98983,0.96303,0.953898
2,buy,0.953898,1.00924,0.97465,0.945983
3,sell,0.945983,0.99396,0.97292,0.962444
4,buy,0.962444,1.07351,0.99074,1.019839
5,sell,1.019839,1.06372,1.04822,1.031770
6,buy,1.031770,1.10320,1.06285,1.040257
7,sell,1.040257,1.08035,1.05234,1.040500
8,buy,1.040500,1.07590,1.05214,1.022787
9,sell,1.022787,1.07299,1.05150,1.040112
10,buy,1.040112,1.10946,1.07031,1.059657


In [817]:
def calc_profit_points(row):
    if row['signal'] == 'buy':
        val = row['close_price'] - row['open_price']
    elif row['signal'] == 'sell':
        val = row['open_price'] - row['close_price']
    else:
        val = 0
    return val

min_mart_factor = 1

def apply_profit_points(dataframe, function):
    dataframe['profit_points'] = dataframe.apply(function, axis=1)
    dataframe['profit_type'] = np.where(dataframe['profit_points'] > 0, 'win', 'loss')
    # dataframe['mart_factor'] = (dataframe['profit_type'].groupby((dataframe['profit_type'] != dataframe['profit_type'].shift()).cumsum()).cumcount() + 1)
    dataframe.reset_index(drop=True, inplace=True)
    
    return dataframe

apply_profit_points(dataframe=deals, function=calc_profit_points)

,signal,open_price,highest,lowest,close_price,profit_points,profit_type
0,sell,0.941181,0.98983,0.96303,0.953898,-0.012717,loss
1,buy,0.953898,1.00924,0.97465,0.945983,-0.007915,loss
2,sell,0.945983,0.99396,0.97292,0.962444,-0.016461,loss
3,buy,0.962444,1.07351,0.99074,1.019839,0.057395,win
4,sell,1.019839,1.06372,1.04822,1.031770,-0.011931,loss
5,buy,1.031770,1.10320,1.06285,1.040257,0.008488,win
6,sell,1.040257,1.08035,1.05234,1.040500,-0.000242,loss
7,buy,1.040500,1.07590,1.05214,1.022787,-0.017712,loss
8,sell,1.022787,1.07299,1.05150,1.040112,-0.017324,loss
9,buy,1.040112,1.10946,1.07031,1.059657,0.019545,win


In [818]:
def calc_martingale_factor(dataframe):
    
    dataframe['profit_code'] = np.where(dataframe['profit_points'] > 0, 0, 1)

    profit_code = list(deals['profit_code'])

    profit_code = pd.Series(profit_code)

    steps = (profit_code.cumsum() * profit_code).diff() # mask out the cumsum where we won [0 1 2 3 0 0 4 5 6 ... ]

    edges = steps < 0 # find where the cumsum steps down -> where we won
    diff_steps = steps[edges].diff() # find the length of each losing streak
    diff_steps[steps[edges].index[0]] = steps[edges][:1] # fix length of the first run which in now NaN
    toss = profit_code.copy() # get a copy of the profit_code series
    toss[edges] = diff_steps # insert the length of the losing streaks into the copy of the profit_code results
    mart_factor = 2 ** (toss).cumsum() # compute the wagers

    result = pd.DataFrame({
        'profit_code': profit_code,
        'toss': toss,
        'runs': toss.cumsum(),
        'mart_factor': mart_factor
        })
    
    result['profit_toss'] = np.where(result['profit_code'] == 0, 'win', 'loss')
    
    result['mart_factor'] = result['mart_factor'].shift(periods=1, fill_value=1)
    
    # drop profit_code from dataframe
    dataframe = dataframe.drop('profit_code', axis=1) 
    
    # add mart_profit to the dataframe
    dataframe['mart_factor']= result['mart_factor']
    
    return dataframe

calc_martingale_factor(dataframe=deals)

,signal,open_price,highest,lowest,close_price,profit_points,profit_type,mart_factor
0,sell,0.941181,0.98983,0.96303,0.953898,-0.012717,loss,1
1,buy,0.953898,1.00924,0.97465,0.945983,-0.007915,loss,2
2,sell,0.945983,0.99396,0.97292,0.962444,-0.016461,loss,4
3,buy,0.962444,1.07351,0.99074,1.019839,0.057395,win,8
4,sell,1.019839,1.06372,1.04822,1.031770,-0.011931,loss,1
5,buy,1.031770,1.10320,1.06285,1.040257,0.008488,win,2
6,sell,1.040257,1.08035,1.05234,1.040500,-0.000242,loss,1
7,buy,1.040500,1.07590,1.05214,1.022787,-0.017712,loss,2
8,sell,1.022787,1.07299,1.05150,1.040112,-0.017324,loss,4
9,buy,1.040112,1.10946,1.07031,1.059657,0.019545,win,8


In [819]:
start_balance = 1000
contract = 100000
min_lot= 0.01
volume = contract * min_lot

def create_profit_values(dataframe):
    # dataframe['lot'] = dataframe['mart_factor'] * min_lot
    dataframe['profit_value'] = dataframe['profit_points'] * volume

    return dataframe

create_profit_values(dataframe=deals)

,signal,open_price,highest,lowest,close_price,profit_points,profit_type,profit_code,profit_value
0,sell,0.941181,0.98983,0.96303,0.953898,-0.012717,loss,1,-12.7167
1,buy,0.953898,1.00924,0.97465,0.945983,-0.007915,loss,1,-7.9152
2,sell,0.945983,0.99396,0.97292,0.962444,-0.016461,loss,1,-16.4609
3,buy,0.962444,1.07351,0.99074,1.019839,0.057395,win,0,57.3949
4,sell,1.019839,1.06372,1.04822,1.031770,-0.011931,loss,1,-11.9310
5,buy,1.031770,1.10320,1.06285,1.040257,0.008488,win,0,8.4875
6,sell,1.040257,1.08035,1.05234,1.040500,-0.000242,loss,1,-0.2425
7,buy,1.040500,1.07590,1.05214,1.022787,-0.017712,loss,1,-17.7122
8,sell,1.022787,1.07299,1.05150,1.040112,-0.017324,loss,1,-17.3242
9,buy,1.040112,1.10946,1.07031,1.059657,0.019545,win,0,19.5455


In [820]:
def calc_min_lot():
    pass

In [821]:
# Visualize Close Price
px.line(df, x='time', y='close')

In [822]:
def find_crossover(fast_sma, prev_fast_sma, slow_sma):
    
    if fast_sma > slow_sma and prev_fast_sma < slow_sma:
        return 'bullish crossover'
    elif fast_sma < slow_sma and prev_fast_sma > slow_sma:
        return 'bearish crossover'
    
    return None


df['crossover'] = np.vectorize(find_crossover)(df['fast_sma'], df['prev_fast_sma'], df['slow_sma'])

# df.dropna(inplace=True)
# signal = df[df['crossover'] == 'bullish crossover'].copy() # => Original
signal = df[(df['crossover'] == 'bullish crossover') | (df['crossover'] == 'bearish crossover')].copy()

signal

,time,open,high,low,close,spread,fast_sma,slow_sma,prev_fast_sma,signal,crossover
247,2022-10-24 08:00:00,0.98465,0.98610,0.98158,0.98340,20,0.984582,0.984495,0.983038,sell,bullish crossover
295,2022-11-03 08:00:00,0.98251,0.98251,0.97465,0.97524,20,0.982018,0.982338,0.984838,buy,bearish crossover
303,2022-11-04 16:00:00,0.98656,0.99396,0.98592,0.99223,20,0.981998,0.981484,0.978516,sell,bullish crossover
564,2023-01-06 04:00:00,1.05242,1.05353,1.05096,1.05133,15,1.052758,1.052970,1.054942,buy,bearish crossover
567,2023-01-06 16:00:00,1.05416,1.06372,1.05001,1.06367,19,1.054446,1.053404,1.052114,sell,bullish crossover
694,2023-02-06 20:00:00,1.07122,1.07363,1.07093,1.07242,15,1.075100,1.075513,1.076300,buy,bearish crossover
844,2023-03-13 20:00:00,1.07377,1.07418,1.07200,1.07273,15,1.071036,1.070691,1.069844,sell,bullish crossover
854,2023-03-15 12:00:00,1.06771,1.06834,1.05214,1.05442,20,1.068516,1.069705,1.072272,buy,bearish crossover
872,2023-03-20 12:00:00,1.06735,1.07299,1.06699,1.07228,19,1.068258,1.067631,1.067240,sell,bullish crossover
1101,2023-05-11 16:00:00,1.09319,1.09323,1.08992,1.09242,8,1.094996,1.095586,1.096154,buy,bearish crossover


In [823]:
# visualize close price
fig = px.line(df, x='time', y=['close', 'fast_sma', 'slow_sma'])

for i, row in signal.iterrows():
    fig.add_vline(x=row.time)
    
fig.show()

In [824]:
# creating backtest and position classes

class Position:
    def __init__(self, open_datetime, open_price, order_type, volume, lot_multiplier, sl, tp):
        self.open_datetime = open_datetime
        self.open_price = open_price
        self.order_type = order_type
        self.lot_multiplier = lot_multiplier
        self.volume = volume
        self.sl = sl
        self.tp = tp
        self.close_datetime = None
        self.close_price = None
        self.profit = None
        self.status = 'open'
        
    def close_position(self, close_datetime, close_price):
        self.close_datetime = close_datetime
        self.close_price = close_price
        self.profit = (self.close_price - self.open_price) * self.volume * self.lot_multiplier if self.order_type == 'buy' \
                                                                        else (self.open_price - self.close_price) * self.volume * self.lot_multiplier
        self.status = 'closed'
        
    def _asdict(self):
        return {
            'open_datetime': self.open_datetime,
            'open_price': self.open_price,
            'order_type': self.order_type,
            'volume': self.volume,
            'lot_multiplier': self.lot_multiplier,
            'sl': self.sl,
            'tp': self.tp,
            'close_datetime': self.close_datetime,
            'close_price': self.close_price,
            'profit': self.profit,
            'status': self.status,
        }
        
        
class Strategy:
    def __init__(self, df, starting_balance, volume, lot_multiplier):
        self.starting_balance = starting_balance
        self.positions = []
        self.lot_multiplier = lot_multiplier
        self.volume = volume
        self.data = df
        
    def get_positions_df(self):
        df = pd.DataFrame([position._asdict() for position in self.positions])
        df['pnl'] = df['profit'].cumsum() + self.starting_balance
        return df
        
    def add_position(self, position):
        self.positions.append(position)
        
        return True
        
# logic
    def run(self):
        for i, data in self.data.iterrows():
            
            if data.crossover == 'bearish crossover':
                for position in self.positions:
                    if position.status == 'open':
                        position.close_position(data.time, data.close)
                self.add_position(Position(data.time, data.close, 'sell', self.volume, 1, 0, 0))
            
            if data.crossover == 'bullish crossover':
                for position in self.positions:
                    if position.status == 'open':
                        position.close_position(data.time, data.close)
                self.add_position(Position(data.time, data.close, 'buy', self.volume, 1, 0, 0))
        
        return self.get_positions_df()

In [825]:
# Lot Size:
# Standard => Volume = 100,000 => Lot = 1.00 => pip Value = 10.00 USD
# Mini Lot => Volume = 10,000 => Lot = 0.10 => pip Value = 1.00 USD
# Micro Lot => Volume = 1,000 => Lot = 0.01 => pip Value = 0.10 USD
# Nano Lot => Volume = 100 => Lot = 0.0001 => pip Value = 10 USD

starting_balance = 1000 
standard_contract = 100000
min_lot = 0.1
lot_multiplier = 2 # 1 if not self.positions or self.positions[-1] > 0 else 2
volume = min_lot * standard_contract * lot_multiplier


In [826]:
sma_crossover_strategy = Strategy(df, starting_balance, volume, lot_multiplier)
deals = sma_crossover_strategy.run()
deals.dropna(inplace=True)
deals

,open_datetime,open_price,order_type,volume,lot_multiplier,sl,tp,close_datetime,close_price,profit,status,pnl
0,2022-10-24 08:00:00,0.98340,buy,20000.0,1,0,0,2022-11-03 08:00:00,0.97524,-163.2,closed,836.8
1,2022-11-03 08:00:00,0.97524,sell,20000.0,1,0,0,2022-11-04 16:00:00,0.99223,-339.8,closed,497.0
2,2022-11-04 16:00:00,0.99223,buy,20000.0,1,0,0,2023-01-06 04:00:00,1.05133,1182.0,closed,1679.0
3,2023-01-06 04:00:00,1.05133,sell,20000.0,1,0,0,2023-01-06 16:00:00,1.06367,-246.8,closed,1432.2
4,2023-01-06 16:00:00,1.06367,buy,20000.0,1,0,0,2023-02-06 20:00:00,1.07242,175.0,closed,1607.2
5,2023-02-06 20:00:00,1.07242,sell,20000.0,1,0,0,2023-03-13 20:00:00,1.07273,-6.2,closed,1601.0
6,2023-03-13 20:00:00,1.07273,buy,20000.0,1,0,0,2023-03-15 12:00:00,1.05442,-366.2,closed,1234.8
7,2023-03-15 12:00:00,1.05442,sell,20000.0,1,0,0,2023-03-20 12:00:00,1.07228,-357.2,closed,877.6
8,2023-03-20 12:00:00,1.07228,buy,20000.0,1,0,0,2023-05-11 16:00:00,1.09242,402.8,closed,1280.4
9,2023-05-11 16:00:00,1.09242,sell,20000.0,1,0,0,2023-07-28 08:00:00,1.09588,-69.2,closed,1211.2


In [827]:
#  Get Columns Titles to check Existing Columns
titles = deals.columns

for title in titles:
 print(title)


open_datetime
open_price
order_type
volume
lot_multiplier
sl
tp
close_datetime
close_price
profit
status
pnl


In [828]:
lot_multiplier = 2
deals['profit_1'] = deals['profit'].shift(1).fillna(0)
deals
# if deals['profit'].shift(-1) > 0 or deals['profit'].shift(-1) == None:
#     deals['lot'] = 0.1
# else:
#     deals['lot'] = 0.1 * lot_multiplier

,open_datetime,open_price,order_type,volume,lot_multiplier,sl,tp,close_datetime,close_price,profit,status,pnl,profit_1
0,2022-10-24 08:00:00,0.98340,buy,20000.0,1,0,0,2022-11-03 08:00:00,0.97524,-163.2,closed,836.8,0.0
1,2022-11-03 08:00:00,0.97524,sell,20000.0,1,0,0,2022-11-04 16:00:00,0.99223,-339.8,closed,497.0,-163.2
2,2022-11-04 16:00:00,0.99223,buy,20000.0,1,0,0,2023-01-06 04:00:00,1.05133,1182.0,closed,1679.0,-339.8
3,2023-01-06 04:00:00,1.05133,sell,20000.0,1,0,0,2023-01-06 16:00:00,1.06367,-246.8,closed,1432.2,1182.0
4,2023-01-06 16:00:00,1.06367,buy,20000.0,1,0,0,2023-02-06 20:00:00,1.07242,175.0,closed,1607.2,-246.8
5,2023-02-06 20:00:00,1.07242,sell,20000.0,1,0,0,2023-03-13 20:00:00,1.07273,-6.2,closed,1601.0,175.0
6,2023-03-13 20:00:00,1.07273,buy,20000.0,1,0,0,2023-03-15 12:00:00,1.05442,-366.2,closed,1234.8,-6.2
7,2023-03-15 12:00:00,1.05442,sell,20000.0,1,0,0,2023-03-20 12:00:00,1.07228,-357.2,closed,877.6,-366.2
8,2023-03-20 12:00:00,1.07228,buy,20000.0,1,0,0,2023-05-11 16:00:00,1.09242,402.8,closed,1280.4,-357.2
9,2023-05-11 16:00:00,1.09242,sell,20000.0,1,0,0,2023-07-28 08:00:00,1.09588,-69.2,closed,1211.2,402.8


In [829]:
#  Get total profit
total_profit = deals['profit'].sum()

total_profit

332.80000000001303

In [830]:
px.line(deals, x='close_datetime', y='pnl')

In [831]:
# visualize close price
fig = px.line(df, x='time', y=['close', 'fast_sma', 'slow_sma'])

for i, row in signal.iterrows():
    fig.add_vline(x=row.time)
    
for i, row in deals[deals['status'] == 'closed'].iterrows():
    
    if row.profit > 0:
        fig.add_shape(type="line",
            x0=row.open_datetime, y0=row.open_price, x1=row.close_datetime, y1=row.close_price,
            line=dict(color="Green",width=3)
                     )
                      
    elif row.profit < 0:
        fig.add_shape(type="line",
            x0=row.open_datetime, y0=row.open_price, x1=row.close_datetime, y1=row.close_price,
            line=dict(color="Red",width=3)
                      )

    
fig.show()

In [832]:
def apply_profit_type(x):
# Finding profit type
  if x['profit'] > 0: 
      return "win"
  elif x['profit'] < 0:
      return "loss"
  else: return 'pass'

deals['profit_type'] = deals.apply(apply_profit_type, axis=1)

deals.dropna(inplace=True)

display(deals)

,open_datetime,open_price,order_type,volume,lot_multiplier,sl,tp,close_datetime,close_price,profit,status,pnl,profit_1,profit_type
0,2022-10-24 08:00:00,0.98340,buy,20000.0,1,0,0,2022-11-03 08:00:00,0.97524,-163.2,closed,836.8,0.0,loss
1,2022-11-03 08:00:00,0.97524,sell,20000.0,1,0,0,2022-11-04 16:00:00,0.99223,-339.8,closed,497.0,-163.2,loss
2,2022-11-04 16:00:00,0.99223,buy,20000.0,1,0,0,2023-01-06 04:00:00,1.05133,1182.0,closed,1679.0,-339.8,win
3,2023-01-06 04:00:00,1.05133,sell,20000.0,1,0,0,2023-01-06 16:00:00,1.06367,-246.8,closed,1432.2,1182.0,loss
4,2023-01-06 16:00:00,1.06367,buy,20000.0,1,0,0,2023-02-06 20:00:00,1.07242,175.0,closed,1607.2,-246.8,win
5,2023-02-06 20:00:00,1.07242,sell,20000.0,1,0,0,2023-03-13 20:00:00,1.07273,-6.2,closed,1601.0,175.0,loss
6,2023-03-13 20:00:00,1.07273,buy,20000.0,1,0,0,2023-03-15 12:00:00,1.05442,-366.2,closed,1234.8,-6.2,loss
7,2023-03-15 12:00:00,1.05442,sell,20000.0,1,0,0,2023-03-20 12:00:00,1.07228,-357.2,closed,877.6,-366.2,loss
8,2023-03-20 12:00:00,1.07228,buy,20000.0,1,0,0,2023-05-11 16:00:00,1.09242,402.8,closed,1280.4,-357.2,win
9,2023-05-11 16:00:00,1.09242,sell,20000.0,1,0,0,2023-07-28 08:00:00,1.09588,-69.2,closed,1211.2,402.8,loss


In [833]:
# Find Initial Deposit = Initial Balance 
deposit = starting_balance
print('deposit = ', deposit)

# find total net profit
total_net_profit = round(deals['profit'].sum())
print('total net profit = ', total_net_profit)

# find gross profit
gross_profit = round(sum(deals[deals['profit_type']=='win']['profit']))
print('gross profit = ', gross_profit)

# find gross loss
gross_loss = round(sum(deals[deals['profit_type']=='loss']['profit']))
print('gross loss = ', gross_loss)

#  find profit factor
profit_factor = gross_profit / gross_loss
print('profit factor = ', profit_factor)

# find largest profit trade
largest_profit_trade = round(deals['profit'].max())
print('largest profit trade = ', largest_profit_trade)

# find largest loss trade
largest_loss_trade = round(deals['profit'].min())
print('largest loss trade = ', largest_loss_trade)

deposit =  1000
total net profit =  333
gross profit =  2720
gross loss =  -2387
profit factor =  -1.1395056556346879
largest profit trade =  1182
largest loss trade =  -366


In [834]:
out = (
    deals.groupby(deals["profit_type"].ne(deals["profit_type"].shift()).cumsum(), as_index=False)
    .agg({"profit_type": "first", "profit": "sum"})
    .rename(columns={"profit": "grand_profit"})
)
display(out)

,profit_type,grand_profit
0,loss,-503.0
1,win,1182.0
2,loss,-246.8
3,win,175.0
4,loss,-729.6
5,win,402.8
6,loss,-257.8
7,win,642.0
8,loss,-353.4
9,win,265.0


In [835]:
# Create Summary DataFrame
summary = pd.DataFrame(columns=['fast_sma',
                                'slow_sma',
                                
                                'deposit',  # deposit
                                'balance',  # balance
                                
                                'net_profit', # total net profit
                                
                                'gross_profit', # sum of win trades
                                'gross_loss', # sum of fail trades
                                'profit_factor', # ratio (%) => (gross profit / gross loss) => one means (profit = loss)
                                
                                'abs_drawdown', # the largest loss is lower than the initial deposit value
                                'max_drawdown', # maximal loss of the local maximum in the deposit currency and in percents of the deposit
                                'rel_drawdown', #  the maximal loss in percents of the maximum equity value and its corresponding money value
                                
                                'total_trades', # count total trades
                                'total_buy', # count total BUY trades
                                'total_sell', # count total SELL trades
                                
                                'max_win', # Largest profit trade
                                'max_fail', # Largest loss trade
                                
                                'min_lot', # initial lot
                                'lot_multiplier',
                                'max_lot', # highest lot
                                
                                'F',
                                'G'])

# balance =  9971.76
# total net profit =  -28.24
# gross profit =  5.43
# gross loss =  -33.67
# profit factor =  0.16
# largest profit trade =  5.2
# largest loss trade =  -2.2
# total trades =  65
# count SELL trades =  24
# count BUY trades =  41
# count WIN trades =  5
# count LOSS trades =  60
# ave_win_value =  1.09
# ave_loss_value =  -0.56

summary

,fast_sma,slow_sma,deposit,balance,net_profit,gross_profit,gross_loss,profit_factor,abs_drawdown,max_drawdown,...,total_trades,total_buy,total_sell,max_win,max_fail,min_lot,lot_multiplier,max_lot,F,G
